In [1]:
import os
from pathlib import Path

from haystack import Pipeline
from haystack.components.converters import TextFileToDocument
from haystack.components.writers import DocumentWriter

from haystack_integrations.document_stores.chroma import ChromaDocumentStore

In [2]:
file_paths = ["../data/parsed_files" / Path(name) for name in os.listdir("../data/parsed_files")]

In [3]:
len(file_paths)

148

In [4]:
# Chroma is used in-memory so we use the same instances in the two pipelines below
document_store = ChromaDocumentStore()

indexing = Pipeline()
indexing.add_component("converter", TextFileToDocument())
indexing.add_component("writer", DocumentWriter(document_store))
indexing.connect("converter", "writer")
indexing.run({"converter": {"sources": file_paths}})

{'writer': {'documents_written': 148}}

In [5]:
from haystack_integrations.components.retrievers.chroma import ChromaQueryTextRetriever
# from haystack.components.generators import HuggingFaceAPIGenerator
from haystack.components.generators import OpenAIGenerator
from haystack.components.builders import PromptBuilder
from haystack.utils import Secret

In [6]:
# MODEL_NAME = "hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0"
MODEL_NAME = "qwen2.5:7b"
llm = OpenAIGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={
        "temperature": 0.8
    }
)

In [5]:
result = llm.run("What is the most interesting thing you know?")
for answer in result["replies"]:
    print(answer)


KeyboardInterrupt



In [6]:
from haystack.components.tools import ToolInvoker
from haystack.tools import create_tool_from_function
from typing import Annotated

def get_current_weather(
    location: Annotated[str, "The city for which to get the weather, e.g. 'San Francisco'"] = "Munich",
    unit: Annotated[str, "The unit for the temperature, e.g. 'celsius'"] = "celsius",
):
  return {"weather": "sunny", "temperature": 21.8, "unit": unit}

weather_tool = create_tool_from_function(get_current_weather)

In [9]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

client = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={
        "temperature": 0.8
    },
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True)
)
response = client.run(
	  [ChatMessage.from_user("What's Natural Language Processing? Be brief.")]
)
print(response)

Natural Language Processing (NLP) is a subfield of artificial intelligence that focuses on the interaction between human language and computers, enabling machines to understand, interpret, and generate human-like text in various applications such as text translation, sentiment analysis, chatbots, and search engines.{'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='Natural Language Processing (NLP) is a subfield of artificial intelligence that focuses on the interaction between human language and computers, enabling machines to understand, interpret, and generate human-like text in various applications such as text translation, sentiment analysis, chatbots, and search engines.')], _name=None, _meta={'model': 'hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0', 'index': 0, 'finish_reason': 'stop', 'completion_start_time': '2025-04-18T05:28:28.121267', 'usage': {}})]}


In [7]:
from haystack.components.builders import ChatPromptBuilder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack import Pipeline
from haystack.utils import Secret

# no parameter init, we don't use any runtime template variables
prompt_builder = ChatPromptBuilder()
llm = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={
        "temperature": 0.8
    },
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True)
)

pipe = Pipeline()
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)
pipe.connect("prompt_builder.prompt", "llm.messages")
location = "Berlin"
messages = [ChatMessage.from_system("Always respond in German even if some input data is in other languages."),
            ChatMessage.from_user("Tell me about {{location}}")]
pipe.run(data={"prompt_builder": {"template_variables":{"location": location}, "template": messages}})

Berlin ist die Hauptstadt von Deutschland und eine moderne Metropole mit einer beeindruckenden Geschichte. Die Stadt kombiniert edle Bauwerke, wie das Reichstag oder der Brandenburger Tor, mit modernen Architekturbeispielen in Neukölln oder Prenzlauer Berg. Berlin ist bekannt für seine Vielfalt an Kunst und Kultur: Museen wie das Museum insel auf Museum Island und die DDR-Museum bieten Einblicke in die deutsche Geschichte, während Veranstaltungsorte wie der HAU und der Theater am Avantgarde-Kollektiv für moderne Performance-Aufführungen sorgen. Die Stadt zählt auch zu den führenden Zentren der Kreativität mit zahlreichen Galerien, Künstlerstudios und Designern. Berlin ist eine Stadt für alle Hobbys: von Spaziergängen in der Tiergarten-Schloßanlage bis hin zu Partytouren im Alexanderplatz-Bereich oder den Prenzlauer Berg-Nachtclubs. Darüber hinaus bietet die U-Bahn-Netz der Stadt eine effiziente Möglichkeit, sich an den Orten zu orientieren, wo man sein Interesse amüsant und unterhaltsa

{'llm': {'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='Berlin ist die Hauptstadt von Deutschland und eine moderne Metropole mit einer beeindruckenden Geschichte. Die Stadt kombiniert edle Bauwerke, wie das Reichstag oder der Brandenburger Tor, mit modernen Architekturbeispielen in Neukölln oder Prenzlauer Berg. Berlin ist bekannt für seine Vielfalt an Kunst und Kultur: Museen wie das Museum insel auf Museum Island und die DDR-Museum bieten Einblicke in die deutsche Geschichte, während Veranstaltungsorte wie der HAU und der Theater am Avantgarde-Kollektiv für moderne Performance-Aufführungen sorgen. Die Stadt zählt auch zu den führenden Zentren der Kreativität mit zahlreichen Galerien, Künstlerstudios und Designern. Berlin ist eine Stadt für alle Hobbys: von Spaziergängen in der Tiergarten-Schloßanlage bis hin zu Partytouren im Alexanderplatz-Bereich oder den Prenzlauer Berg-Nachtclubs. Darüber hinaus bietet die U-Bahn-Netz der Stadt eine e

In [11]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

client = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={
        "temperature": 0.8
    },
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True)
)
response = client.run(
    [ChatMessage.from_user("What's Natural Language Processing? Be brief.")]
)
print(response)

Natural Language Processing (NLP) is a subfield of artificial intelligence, computer science, and linguistics focused on the interaction between human language and computers. It enables machines to understand, interpret, and generate human language in both spoken and written forms.{'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='Natural Language Processing (NLP) is a subfield of artificial intelligence, computer science, and linguistics focused on the interaction between human language and computers. It enables machines to understand, interpret, and generate human language in both spoken and written forms.')], _name=None, _meta={'model': 'hf.co/IlyaGusev/saiga_nemo_12b_gguf:Q8_0', 'index': 0, 'finish_reason': 'stop', 'completion_start_time': '2025-04-18T05:35:31.934860', 'usage': {}})]}


In [8]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

client = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={
        "temperature": 0.8
    },
    streaming_callback=print_streaming_chunk
)
response = client.run(
    [ChatMessage.from_user("What's Natural Language Processing? Be brief.")]
)

Natural Language Processing (NLP) is a branch of artificial intelligence that focuses on the interaction between computers and human language. It involves programming computers to process, understand, and generate human language in a useful and intelligent way. NLP techniques can be used for tasks such as translation, sentiment analysis, text summarization, and chatbot responses.

In [9]:
def get_current_weather(location: str, unit: str = "celsius"):
  ## Do something
  return {"weather": "sunny", "temperature": 21.8, "unit": unit}

available_functions = {
  "get_current_weather": get_current_weather
}

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "unit"],
            },
        }
    }
]


In [11]:
from haystack.dataclasses import ChatMessage

messages = []
messages.append(ChatMessage.from_system("Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."))
messages.append(ChatMessage.from_user("What's the weather like in Berlin?"))

client = OpenAIChatGenerator(
    model=MODEL_NAME,
    api_key=Secret.from_token("ollama"),
    api_base_url="http://localhost:11434/v1",
    generation_kwargs={
        "temperature": 0.8
    },
    streaming_callback=print_streaming_chunk
)
response = client.run(
    messages=messages,
    generation_kwargs={"tools":tools}
)

In [12]:
response

{'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[ToolCall(tool_name='get_current_weather', arguments={'location': 'Berlin, Germany', 'unit': 'celsius'}, id='call_6l4u39rc')], _name=None, _meta={'model': 'qwen2.5:7b', 'index': 0, 'finish_reason': 'tool_calls', 'completion_start_time': '2025-04-18T06:21:19.920241', 'usage': {}})]}

In [13]:
messages.append(response["replies"][0])

In [14]:
import json

function_call = json.loads(response["replies"][0].content)[0]
function_name = function_call["function"]["name"]
function_args = json.loads(function_call["function"]["arguments"])
print("function_name:", function_name)
print("function_args:", function_args)

AttributeError: The `content` attribute of `ChatMessage` has been removed. Use the `text` property to access the textual value. For more information about the new API and how to migrate, see the documentation: https://docs.haystack.deepset.ai/docs/chatmessage

In [15]:
response = client.run(
    messages=messages,
    generation_kwargs={"tools":tools}
)

In [16]:
response

{'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[], _name=None, _meta={'model': 'qwen2.5:7b', 'index': 0, 'finish_reason': 'stop', 'completion_start_time': '2025-04-18T06:25:03.722324', 'usage': {}})]}

In [11]:
prompt = """
Ответьте на запрос на основе предоставленного контекста.
Если контекст не содержит ответа, скажите «Ответ не найден».
Контекст:
{% for doc in documents %}
  {{ doc.content }}
{% endfor %}
Запрос: {{query}}
Ответ:
"""
prompt_builder = PromptBuilder(template=prompt)

retriever = ChromaQueryTextRetriever(document_store)

querying = Pipeline()
querying.add_component("retriever", retriever)
querying.add_component("prompt_builder", prompt_builder)
querying.add_component("llm", llm)

querying.connect("retriever.documents", "prompt_builder.documents")
querying.connect("prompt_builder", "llm")

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


🚅 Components
  - retriever: ChromaQueryTextRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [13]:
%%time

query = "Как менялось предназначение здания музея Прадо на протяжении истории, и какие ключевые события повлияли на его становление?"
results = querying.run({
    "retriever": {"query": query, "top_k": 3},
    "prompt_builder": {"query": query},
})

In [14]:
print(results["llm"]["replies"][0])


Здание музея Прадо прошло значительные изменения своего предназначения за более чем пятивековую историю. Вот основные этапы его развития: 1503 год - основание королевских коллекций, предшествовавших музею (королевская библиотека и художественные коллекции).

Ключевые события и факторы влияния на становление музея Прадо:
- Консолидация художественных коллекций испанскими монархами в XVI-XVII веках
- Строительство Паласио дель Буэнос Айрес (1738) как резиденции Испанской королевской семьи, где размещены основные музейные собрания
- Оформление статуса музея королевских коллекций после смерти короля Карла III (1792)
- Реорганизация и расширение коллекции во время правления Филиппа V и последующих монархов
- Признание Прадо важным культурным центром Европы в XIX веке
- Официальное открытие музея в 1819 году как современного учреждения, собирающего произведения искусства

На протяжении этих этапов происходило постепенное формирование музея Прадо как одного из ведущих мировых учреждений в об

Изначально на месте Прадо в XV веке находился охотничий домик короля Энрике III, построенный в 1405 году. Позже, в 1543 году, Карл V Габсбургский начал возводить здесь дворец для своего сына Филиппа II. При Филиппе II во дворце разместилась художественная галерея, но в 1604 году она сгорела и была восстановлена по приказу Филиппа III.

В эпоху Просвещения (XVIII век) Карл III Бурбонский превратил парк Прадо в научный центр, разбив ботанический сад и построив здание для Академии наук. Однако это здание так и не стало академией — вместо него в 1819 году открылся музей Прадо по указу Фердинанда VII. Таким образом, от охотничьего домика до королевского дворца, а затем — до всемирно известного музея, Прадо прошёл долгий путь, отражающий изменения в испанской культуре и истории.

In [15]:
results

{'llm': {'replies': ['\nЗдание музея Прадо прошло значительные изменения своего предназначения за более чем пятивековую историю. Вот основные этапы его развития: 1503 год - основание королевских коллекций, предшествовавших музею (королевская библиотека и художественные коллекции).\n\nКлючевые события и факторы влияния на становление музея Прадо:\n- Консолидация художественных коллекций испанскими монархами в XVI-XVII веках\n- Строительство Паласио дель Буэнос Айрес (1738) как резиденции Испанской королевской семьи, где размещены основные музейные собрания\n- Оформление статуса музея королевских коллекций после смерти короля Карла III (1792)\n- Реорганизация и расширение коллекции во время правления Филиппа V и последующих монархов\n- Признание Прадо важным культурным центром Европы в XIX веке\n- Официальное открытие музея в 1819 году как современного учреждения, собирающего произведения искусства\n\nНа протяжении этих этапов происходило постепенное формирование музея Прадо как одного и

In [16]:
import rich
rich.print(results)

{
    'llm': {
        'replies': [
            '\nЗдание музея Прадо прошло значительные изменения своего предназначения за более чем пятивековую 
историю. Вот основные этапы его развития: 1503 год - основание королевских коллекций, предшествовавших музею 
(королевская библиотека и художественные коллекции).\n\nКлючевые события и факторы влияния на становление музея 
Прадо:\n- Консолидация художественных коллекций испанскими монархами в XVI-XVII веках\n- Строительство Паласио дель
Буэнос Айрес (1738) как резиденции Испанской королевской семьи, где размещены основные музейные собрания\n- 
Оформление статуса музея королевских коллекций после смерти короля Карла III (1792)\n- Реорганизация и расширение 
коллекции во время правления Филиппа V и последующих монархов\n- Признание Прадо важным культурным центром Европы в
XIX веке\n- Официальное открытие музея в 1819 году как современного учреждения, собирающего произведения 
искусства\n\nНа протяжении этих этапов происходило постепенное формирование музея Прадо как одного из ведущих 
мировых учреждений в области изобразительного искусства. Изменения предназначения здания отражают эволюцию музейной
практики и понимания роли музеев в обществе.'
        ],
        'meta': [
            {
                'model': 'hf.co/IlyaGusev/saiga_yandexgpt_8b_gguf:Q6_K',
                'index': 0,
                'finish_reason': 'stop',
                'usage': {
                    'completion_tokens': 229,
                    'prompt_tokens': 2048,
                    'total_tokens': 2277,
                    'completion_tokens_details': None,
                    'prompt_tokens_details': None
                }
            }
        ]
    }
}

In [18]:
%%time

query = "Какие особенности картины Питера Брейгеля Старшего «Жатва» отражают его уникальный стиль и влияние эпохи Возрождения?"
results = querying.run({
    "retriever": {"query": query, "top_k": 3},
    "prompt_builder": {"query": query},
})


KeyboardInterrupt



Картина «Жатва» сочетает в себе средневековую традицию изображения сезонных работ с ренессансными новациями. Брейгель, вдохновленный итальянской живописью, переработал её элементы, создав монументальный пейзаж с панорамным охватом крестьянской жизни. Его уникальность проявляется в детализированном изображении сельского труда и отдыха, а также в гармоничном равновесии природы и человека. В отличие от современников, Брейгель уделял особое внимание крестьянской теме, передавая её с симпатией и восхищением, что отражает его гуманистический взгляд и прозвище «Мужицкий». Кроме того, картина подчеркивает философскую идею договора между человеком и природой (Богом), характерную для мировоззрения художника.